<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/2025-%20Additional%20Results%20-%20with%20seeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import random
import numpy as np
import tensorflow as tf

# -------------------------------------------------------------------
# Seed control
def set_seed(seed=0):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    # uncomment for full determinism (slower but stricter)
    # os.environ['TF_DETERMINISTIC_OPS'] = '1'
# -------------------------------------------------------------------


import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go
import keras
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import saving
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import Callback
import plotly
from keras import losses
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import scipy.stats as stats
import pylab as pl


##### Take one dataset at a time
#train_data = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy')
#train_data = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy')
train_data = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/multivariate_long_sequences-TRAIN-DATA-FullSearch-VAR.npy')


n_seq = train_data.shape[0]
window_size = train_data.shape[1]
n_features = train_data.shape[2]

##For VAR
#TestSample = 10

##For DyforGP
TestSample = 1


##Get large set of subsequence from previous steps (VAE)... this is not labelled yet.
#results1=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated_large_subsquence2_data_V2.npy')
#results1=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/generated_large_subsquence2_data.npy')
results1=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated_large_subsquence2_data.npy')

x=results1



# ================== EXPERIMENT RUNNER (wraps your existing code) ==================

from statsmodels.tsa.api import VAR

def run_experiment(TestSample=1, seed=0):
    """Run one full experiment with fixed seed, return summary metric(s)."""

    set_seed(seed)   # <---- ensure reproducibility for this run

    x_3d = x.reshape((x.shape[0], window_size, n_features))
    n_future = 1
    K = window_size

    import random
    rd = list()
    rd = random.sample(range(1, x_3d.shape[0]), TestSample)

    best_window_for_long_seq = list()
    rmse_list = list()

    for i in rd:
        rmse_list.clear()
        for k in range(2,(round(K))):
            cur_seq = x_3d[i]
            df = pd.DataFrame(cur_seq, columns=[f'V{j+1}' for j in range(n_features)])
            df_train, df_test = df[0:-n_future], df[-n_future:]
            model = VAR(df_train)
            try:
                model_fitted1 = model.fit(k)
                forecast_input1 = df_train.values[-k:]
                fc1 = model_fitted1.forecast(y=forecast_input1, steps=n_future)
                df_forecast1 = pd.DataFrame(fc1, index=df.index[-n_future:], columns=df.columns)
                mse = mean_squared_error(df_test['V1'], df_forecast1['V1'].values)
                rmse_list.append(mse)
            except:
                rmse_list.append(99999)

        # For this i, find minimum rmse for all sliding window, and corresponding sw size
        min_index = rmse_list.index(min(rmse_list))
        min_sw = min_index + 2
        best_window_for_long_seq.append(min_sw)

    # Return something measurable
    return np.mean(rmse_list), np.std(rmse_list), best_window_for_long_seq



# ================== MULTI-SEED LOOP ==================
seeds = [0, 1, 2, 3, 4]
results = []

for s in seeds:
    mean_rmse, std_rmse, best_windows = run_experiment(TestSample=1, seed=s)
    print(f"Seed={s}: mean RMSE={mean_rmse:.4f}, std={std_rmse:.4f}")
    results.append(mean_rmse)

print("\n=== Final Robustness Report ===")
print("RMSE mean ± std:", np.mean(results), np.std(results))


Seed=0: mean RMSE=7142.7878, std=25753.6796
Seed=1: mean RMSE=7142.7871, std=25753.6797
Seed=2: mean RMSE=7142.7857, std=25753.6801
Seed=3: mean RMSE=7142.7857, std=25753.6801
Seed=4: mean RMSE=7142.7857, std=25753.6801

=== Final Robustness Report ===
RMSE mean ± std: 7142.786417000086 0.0008854613993655869


In [ ]:
from google.colab import drive
drive.mount('/content/drive')